Connecting to the OpenAI

In [ ]:
# imports

import os
import requests
from  dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
# load the environment variables from the .env file

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting noteboo")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [ ]:
openai = OpenAI()

# If this doesn't work, try Kernal menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions.

In [ ]:
# To give upu a preview  -- callig OpenAI with these messages is this easy. 

# Some websites need you to use proper headers when fetching them:

headers = {
    "User-Agent": "MOzilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome//117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# lets try one out. Change the website and add print statements to follow along.

cnn = Website("https://www.cnn.com/")
print(cnn.title)
print(cnn.text)

#Type of prompts:

#A system prompt that tells them what task they are performimg and what tone they should use

#A user prompt -- the conversation starter that they should reply to


In [ ]:
# Define our system prompt 
system_prompt = "You are an assistent that analyze the contents of a website \
and provide a short summary, ignoring text that might be navigation related. \
Respond in markdown in english language"

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\n The contents of this website is as folows; \
    please provide a short summary of the website in markdown. \
    If it includes news or announcements, then summarize these too. \n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(cnn))

In [ ]:
messages = [
    {"role": "system", "content": "You are snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [ ]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

In [ ]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
messages_for(cnn)

In [ ]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://www.cnn.com/")

In [ ]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://www.cnn.com/")

In [ ]:
display_summary("https://anthropic.com")

In [ ]:
display_summary("https://deccanherald.com")

**Try new Exercise**

In [ ]:
# Step 1: Create your prompts

system_prompt = "You are a good friend that gives me the information to be a good programmer \
Respond in the markdown."
user_prompt = """
    I am a fresh graduate from college completed MS in computer science.
    Provide me short summary on the daily activities I need to be a successfull in career.
"""

# Step 2: Make the messages list

messages = [
    {"role": "system", "content": "You are a good friend and a well wisher"},
    {"role": "user", "content": "Provide me short summary on the daily activities I need to be a successfull in career"}
] # fill this in

# Step 3: Call OpenAI

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)

# Step 4: print the result

print(response.choices[0].message.content)